<a href="https://www.kaggle.com/code/dakshbathla/mlpr-preprocessing-random?scriptVersionId=239474980" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/influencer-dataset/influencers_data.csv
/kaggle/input/influencer-dataset/brands_data.csv
/kaggle/input/influencer-dataset/final_dataset.csv
/kaggle/input/influencer-dataset/dataset.csv


In [4]:
import pandas as pd
selected_columns=["taken_at_timestamp","dimensions.height","dimensions.width","edge_media_preview_like.count","edge_media_to_caption.edges","edge_media_to_comment.count","edge_media_to_comment.edges","edge_media_to_sponsor_user.edges","edge_media_to_tagged_user.edges","location.name","owner.full_name","owner.is_verified","owner.username"]
df= pd.read_csv('/kaggle/input/influencer-dataset/dataset.csv', usecols=selected_columns)

colnames = list(df.columns)
print(colnames)




<ipython-input-4-703fe9ef4b81>:3: DtypeWarning: Columns (8,11,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('/kaggle/input/influencer-dataset/dataset.csv', usecols=selected_columns)


['taken_at_timestamp', 'dimensions.height', 'dimensions.width', 'edge_media_preview_like.count', 'edge_media_to_caption.edges', 'edge_media_to_comment.count', 'edge_media_to_comment.edges', 'edge_media_to_sponsor_user.edges', 'edge_media_to_tagged_user.edges', 'location.name', 'owner.full_name', 'owner.is_verified', 'owner.username']


In [5]:
import pandas as pd

# Load your dataset (assuming you've already done this)
# df = pd.read_csv('/kaggle/input/your-dataset-name.csv', usecols=selected_columns)

# Filter rows where 'edge_media_to_sponsor_user.edges' starts with "[{'node': {'sponsor':"
df_filtered = df[df['edge_media_to_sponsor_user.edges'].astype(str).str.startswith("[{'node': {'sponsor':")]

# Display the first few rows to verify
print(df_filtered.head())


    taken_at_timestamp dimensions.height dimensions.width  \
122         1542115456              1350             1080   
140         1554418320              1080             1080   
580         1536073523              1350             1080   
600         1526567024              1350             1080   
710         1536718157              1350             1080   

    edge_media_preview_like.count  \
122                          9516   
140                          3556   
580                          3167   
600                           343   
710                          1890   

                           edge_media_to_caption.edges  \
122  [{'node': {'text': 'VÍDEO NOVO no canal! O que...   
140  [{'node': {'text': 'Coelhinho da Páscoa (marid...   
580  [{'node': {'text': 'Most of you have been foll...   
600  [{'node': {'text': '#NoNewFriends Tag your bes...   
710  [{'node': {'text': "Playin' knit cool 🍂 Sharin...   

    edge_media_to_comment.count  \
122                       

In [6]:
# Count the total number of matching rows
total_matching_values = df['edge_media_to_sponsor_user.edges'].astype(str).str.startswith("[{'node': {'sponsor':").sum()

print("Total matching values:", total_matching_values)


Total matching values: 4087


In [7]:
import re

# Function to extract username
def extract_username(value):
    match = re.search(r"username':\s*'([^']+)'", str(value))
    return match.group(1) if match else None

# Apply the function to extract usernames safely
df_filtered.loc[:, 'sponsor_user_username'] = df_filtered['edge_media_to_sponsor_user.edges'].apply(extract_username)

# Display the first few extracted values
print(df_filtered[['edge_media_to_sponsor_user.edges', 'sponsor_user_username']].head(10))


                       edge_media_to_sponsor_user.edges sponsor_user_username
122   [{'node': {'sponsor': {'id': '5435191389', 'us...      drogariasaopaulo
140   [{'node': {'sponsor': {'id': '241178762', 'use...       uniquemonbijoux
580   [{'node': {'sponsor': {'id': '1006338238', 'us...       homesensecanada
600   [{'node': {'sponsor': {'id': '289976614', 'use...               zalando
710   [{'node': {'sponsor': {'id': '1298729246', 'us...                 stage
806   [{'node': {'sponsor': {'id': '7165166824', 'us...        outletdelcorso
3250  [{'node': {'sponsor': {'id': '27567714', 'user...                iorane
3658  [{'node': {'sponsor': {'id': '8596382683', 'us...              shopiixc
4428  [{'node': {'sponsor': {'id': '15638867', 'user...         veuveclicquot
4698  [{'node': {'sponsor': {'id': '196386802', 'use...    constance_calcados


<ipython-input-7-ddede42297d6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'sponsor_user_username'] = df_filtered['edge_media_to_sponsor_user.edges'].apply(extract_username)


In [ ]:
total_usernames = df_filtered['sponsor_user_username'].count()
print("Total extracted usernames:", total_usernames)


In [8]:
import re

# Function to extract full name
def extract_full_name(value):
    match = re.search(r"full_name':\s*'([^']+)'", str(value))
    return match.group(1) if match else None

# Apply the function only to rows where sponsor_user_username is not null
df_filtered.loc[:, 'sponsor_user_full_name'] = df_filtered['edge_media_to_tagged_user.edges'].apply(extract_full_name)

# Display results
print(df_filtered[['sponsor_user_username', 'edge_media_to_tagged_user.edges', 'sponsor_user_full_name']].head(10))

# Count total extracted full names
total_full_names = df_filtered['sponsor_user_full_name'].count()
print("Total extracted full names:", total_full_names)


     sponsor_user_username                    edge_media_to_tagged_user.edges  \
122       drogariasaopaulo  [{'node': {'user': {'full_name': 'Vichy Brasil...   
140        uniquemonbijoux  [{'node': {'user': {'full_name': 'Maysa Leão ✨...   
580        homesensecanada  [{'node': {'user': {'full_name': 'Style at Hom...   
600                zalando  [{'node': {'user': {'full_name': 'ALLSAINTS', ...   
710                  stage  [{'node': {'user': {'id': '1298729246', 'usern...   
806         outletdelcorso  [{'node': {'user': {'full_name': 'OUTLET DEL C...   
3250                iorane                                                 []   
3658              shopiixc  [{'node': {'user': {'full_name': 'Erika Munro ...   
4428         veuveclicquot  [{'node': {'user': {'full_name': 'Veuve Clicqu...   
4698    constance_calcados  [{'node': {'user': {'full_name': 'Constance Ca...   

              sponsor_user_full_name  
122                     Vichy Brasil  
140   Maysa Leão ✨ Marketing D

<ipython-input-8-04d59346889d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'sponsor_user_full_name'] = df_filtered['edge_media_to_tagged_user.edges'].apply(extract_full_name)


In [9]:
# Filter rows where sponsor_user_full_name starts with sponsor_user_username
matching_names = df_filtered[df_filtered.apply(lambda row: str(row['sponsor_user_full_name']).startswith(str(row['sponsor_user_username'])), axis=1)]

# Display results
print(matching_names[['sponsor_user_username', 'sponsor_user_full_name']].head(10))

# Count total matches
total_matches = matching_names.shape[0]
print("Total matching full names:", total_matches)


       sponsor_user_username sponsor_user_full_name
12235                 veryuk                 veryuk
12531                 veryuk                 veryuk
24787                 boohoo             boohoo.com
104884  innovativespiritwear   innovativespiritwear
117351              niveausa               niveausa
134706             babiesrus              babiesrus
177918          invisibobble  invisibobble Official
191943             lululemon              lululemon
212140             pampersde              pampersde
233482                veryuk                 veryuk
Total matching full names: 77


In [10]:
# Define how many starting characters to check (e.g., first 3 characters)
N = 3  

# Filter rows where the first N letters of the username appear in the full name
matching_names = df_filtered[df_filtered.apply(lambda row: str(row['sponsor_user_full_name']).lower().find(str(row['sponsor_user_username'])[:N].lower()) != -1, axis=1)]

# Display results
print(matching_names[['sponsor_user_username', 'sponsor_user_full_name']].head(10))

# Count total matches
total_matches = matching_names.shape[0]
print("Total matching full names:", total_matches)


     sponsor_user_username sponsor_user_full_name
580        homesensecanada          Style at Home
806         outletdelcorso       OUTLET DEL CORSO
4428         veuveclicquot     Veuve Clicquot USA
4698    constance_calcados     Constance Calçados
6047        huggies_france                Huggies
7140             marshalls              Marshalls
7343             garnierfr     garnierdeutschland
7749      dynamiteclothing      Dynamite Clothing
7757        comino.couture  Comino Couture London
7785           teamiblends          Teami™ Blends
Total matching full names: 1960


In [11]:
# Extract the full names into a list
matching_full_names = matching_names['sponsor_user_full_name'].dropna().tolist()



In [12]:
import pandas as pd

# Load the dataset
brands_df = pd.read_csv('/kaggle/input/influencer-dataset/brands_data.csv')


# Display the first few rows
print(brands_df.head())


                            Name  Followers  Followees   Posts  \
0              Organic Care Baby      559.0      111.0    51.0   
1                            아미니    21189.0     3134.0   150.0   
2    THE ABNORMAL BEAUTY COMPANY   368105.0       13.0   850.0   
3      IWC Schaffhausen | Italia    12315.0      106.0   689.0   
4  Four Seasons Resort Marrakech    23876.0      343.0  1325.0   

                                                 URL  \
0  http://naturesorganics.com.au/our-products/chi...   
1                         http://www.aminigoods.com/   
2                             http://www.deciem.com/   
3                             http://www.iwc.com/it/   
4                         http://bit.ly/FSWOA19_prop   

                                      Category  \
0  Personal Goods & General Merchandise Stores   
1  Personal Goods & General Merchandise Stores   
2                        Food & Personal Goods   
3  Personal Goods & General Merchandise Stores   
4       Transpor

<ipython-input-12-0f199744ec13>:4: DtypeWarning: Columns (3,4,5,6,7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  brands_df = pd.read_csv('/kaggle/input/influencer-dataset/brands_data.csv')
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [13]:
# Filter rows where 'name' is in the list of matching full names
matching_brands = brands_df[brands_df['Name'].isin(matching_full_names)]

# Display results
print(matching_brands.head())

# Count total matches
total_matches = matching_brands.shape[0]
print("Total matching names in brands_data.csv:", total_matches)


                   Name  Followers  Followees   Posts  \
38   Smashbox Cosmetics  4309913.0      508.0  3398.0   
41            Rosefield     4817.0        1.0     1.0   
46                 MVMT   432617.0      494.0  1147.0   
99             Oriflame   386949.0      150.0   867.0   
242        Natural Mojo    96619.0     4732.0  2076.0   

                                        URL  \
38                     http://smashbox.com/   
41        https://www.rosefieldwatches.com/   
46   http://mvmtwatch.co/mvmtforherhomepage   
99            http://bit.ly/YouTubeOriflame   
242          https://www.naturalmojo.de/now   

                                        Category  \
38   Personal Goods & General Merchandise Stores   
41                                           NaN   
46                         Food & Personal Goods   
99   Personal Goods & General Merchandise Stores   
242  Personal Goods & General Merchandise Stores   

                                                   Bio       

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [ ]:
# Define the number of starting characters to match
N = 5

# Function to check if the first N letters of any name in matching_full_names appear in brands_df['name']
def is_partial_match(brand_name):
    brand_name = str(brand_name).lower()  # Convert to lowercase for case-insensitive comparison
    return any(str(full_name).lower().startswith(brand_name[:N]) for full_name in matching_full_names)

# Apply the function to filter matching names
matching_brands = brands_df[brands_df['Name'].apply(is_partial_match)]

# Display results

# Count total matches
total_matches = matching_brands.shape[0]
print("Total matching names in brands_data.csv:", total_matches)


In [14]:
import pandas as pd
import random

# Define the number of starting characters to match
N = 5  

# Load extracted influencer full names
matching_full_names = df_filtered['sponsor_user_full_name'].dropna().tolist()  # From previous filtering


# Function to check if an influencer name matches a brand name
def get_matching_brands(influencer_name):
    influencer_name = str(influencer_name).lower()  # Convert to lowercase
    matched_brands = [brand for brand in brands_df['Name'] if str(brand).lower().startswith(influencer_name[:N])]
    return matched_brands if matched_brands else None

# Create a dictionary to store matches
matches_dict = {}

# Loop through all extracted influencer full names and store matched brands
for influencer in matching_full_names:
    matched_brands = get_matching_brands(influencer)
    if matched_brands:
        matches_dict[influencer] = matched_brands  # Store influencer → matched brands

# Convert dictionary to DataFrame
matches_df = pd.DataFrame(list(matches_dict.items()), columns=['Influencer Full Name', 'Matched Brands'])

# Count total matches
total_matches = matches_df.shape[0]
print("Total matching influencers:", total_matches)

# Select 10-15 random samples to print
sample_size = min(15, len(matches_df))  # Ensures we don’t exceed available matches
sampled_matches = matches_df.sample(n=sample_size, random_state=42)  # Random selection

# Print selected matches
print("\nShowing 10-15 sample matches:\n")
for index, row in sampled_matches.iterrows():
    print(f"Influencer: {row['Influencer Full Name']} → Matched Brands: {', '.join(row['Matched Brands'])}")


Total matching influencers: 1784

Showing 10-15 sample matches:

Influencer: Nespresso Italia → Matched Brands: Nespresso, Nespresso UK, nespressousa, Nespresso Italia, Nespresso Canada
Influencer: Naturaverde Bio → Matched Brands: Naturino, Natural Mojo, Naturalizer, NatureWell Beauty, NATURALLY WHITE® Official, Natural Delights Medjool Dates, NATURAL JOE | JOESCRUB, Nature_moi, Natural Skin Care by: Kashae, Nature Made, Natural Fit Designs, Natural Vitality®, Nature's Way Australia, Natural Mojo, Natura, Natural Shilajit Resin, Naturamla, Nature's Fare Markets, Natural Kitchen Restaurant Bar, Nature's Own, naturesfabrics, NatureSweet Tomatoes, Nature's Recipe, Natural Cycles, Nature's Bounty, Natural & Organic Skincare, NATURAL SPA FACTORY, Nature Rocket®, Naturally White OC, Natural Mojo, Nature Republic USA Official, Nature's Miracle, NatureBox, Nature Gnaws, Natural Mojo Polska, Naturhouse Polska, NaturallyCurly.com, Natureshoes, Nature et Découvertes, Naturalicious - Natural Beau

In [ ]:
import pandas as pd
import random

# Define the number of starting characters for partial match
N_PARTIAL = 5

# Load extracted influencer full names
matching_full_names = df_filtered['sponsor_user_full_name'].dropna().tolist()  # From previous filtering

# Load brand data


# Convert brand names to lowercase for efficient lookup
brand_names_set = set(brands_df['Name'].str.lower())  # Convert to a set for fast lookup

# Function to get matching brands
def get_matching_brands(influencer_name):
    influencer_lower = influencer_name.lower()
    
    # 1. **Check for perfect match first**
    if influencer_lower in brand_names_set:
        return [influencer_name]  # Exact match found, return only this
    
    # 2. **If no perfect match, apply partial matching with N_PARTIAL=6**
    matched_brands = [brand for brand in brands_df['Name'] if str(brand).lower().startswith(influencer_lower[:N_PARTIAL])]
    
    return matched_brands if matched_brands else None

# Create a dictionary to store matches
matches_dict = {}

# Loop through all extracted influencer full names and store matched brands
for influencer in matching_full_names:
    matched_brands = get_matching_brands(influencer)
    if matched_brands:
        matches_dict[influencer] = matched_brands  # Store influencer → matched brands

# Convert dictionary to DataFrame
matches_df = pd.DataFrame(list(matches_dict.items()), columns=['Influencer Full Name', 'Matched Brands'])

# Count total matches
total_matches = matches_df.shape[0]
print("Total matching influencers:", total_matches)

# Select 10-15 random samples to print
sample_size = min(15, len(matches_df))  # Ensures we don’t exceed available matches
sampled_matches = matches_df.sample(n=sample_size, random_state=42)  # Random selection

# Print selected matches
print("\nShowing 10-15 sample matches:\n")
for index, row in sampled_matches.iterrows():
    print(f"Influencer: {row['Influencer Full Name']} → Matched Brands: {', '.join(row['Matched Brands'])}")


In [15]:
import pandas as pd

# Load needed data
final_df = pd.read_csv('/kaggle/input/influencer-dataset/final_dataset.csv')

# Use the previously constructed matches_dict (from your code)
# matches_dict = {influencer_full_name: [matched_brand1, matched_brand2, ...]}

# Step 1: Create a mapping from brand name to category
brand_to_category = brands_df.set_index('Name')['Category'].to_dict()

# Step 2: Create final records for each influencer → brands → category
final_records = []

# We'll map full names back to influencer usernames using df_filtered (which has sponsor_user_full_name and owner.username)
full_name_to_username = df_filtered[['sponsor_user_full_name', 'owner.username']].dropna().drop_duplicates()

# Turn it into a dictionary for fast lookup
full_to_user_map = dict(zip(full_name_to_username['sponsor_user_full_name'], full_name_to_username['owner.username']))

# Loop over all matched full names and matched brand names
for full_name, matched_brands in matches_dict.items():
    influencer_username = full_to_user_map.get(full_name)
    
    if influencer_username and matched_brands:
        for brand in matched_brands:
            category = brand_to_category.get(brand, 'Unknown')
            final_records.append({
                'Influencer Username': influencer_username,
                'Brand Name': brand,
                'Brand Category': category
            })

# Step 3: Create final DataFrame
final_df_result = pd.DataFrame(final_records)

# Optional: View a sample
print("\n✅ Final DataFrame Sample:\n")
print(final_df_result.head(1000))

# Optional: Save as CSV
# final_df_result.to_csv("influencer_brand_category.csv", index=False)


<ipython-input-15-79f33a0af4cc>:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  final_df = pd.read_csv('/kaggle/input/influencer-dataset/final_dataset.csv')



✅ Final DataFrame Sample:

    Influencer Username         Brand Name  \
0              jadeseba        Vichy Karto   
1              jadeseba    Vichy Singapore   
2              jadeseba     VICHY Bulgaria   
3              jadeseba       Vichy Canada   
4              jadeseba  VICHY Deutschland   
..                  ...                ...   
995          makaiacarr        Hello Molly   
996          makaiacarr   HelloBody Italia   
997          makaiacarr       Hello Nature   
998          makaiacarr          HELLO! UK   
999          makaiacarr    HELLO! Pakistan   

                                  Brand Category  
0                                            NaN  
1    Personal Goods & General Merchandise Stores  
2    Personal Goods & General Merchandise Stores  
3    Personal Goods & General Merchandise Stores  
4    Personal Goods & General Merchandise Stores  
..                                           ...  
995  Personal Goods & General Merchandise Stores  
996  Person

In [ ]:
print("\n🔢 Total influencer-brand matches:", final_df_result.shape[0])


In [ ]:
final_df_result.head

In [ ]:
df_filtered.head

In [16]:
influencers_df = pd.read_csv('/kaggle/input/influencer-dataset/influencers_data.csv')


In [ ]:
print("📄 Total entries in influencers_df:", influencers_df.shape[0])


In [ ]:
final_df_result.columns

In [ ]:
# Print column names of final_df_result to see what columns exist
print(df_filtered.columns)


In [17]:
# Get the set of usernames from df_filtered
sponsored_usernames = set(df_filtered['owner.username'].dropna())

# Create a new column in influencers_df
influencers_df['sponsorship_final'] = influencers_df['User'].apply(lambda x: 1 if x in sponsored_usernames else 0)

# Show a sample
print(influencers_df[['User', 'sponsorship_final']].head())

# Optional: Count how many are 1s
print("\n✅ Total influencers with sponsorship_final = 1:", influencers_df[influencers_df['sponsorship_final'] == 1].shape[0])


                      User  sponsorship_final
0   silviatodoloquemellama                  0
1  peppermintandgrapefruit                  0
2           rachelclindsay                  0
3      jaquelinesantosblog                  0
4           skincarebysean                  0

✅ Total influencers with sponsorship_final = 1: 1690


In [ ]:
final_df_result.head(20)

In [ ]:
df_filtered.columns



In [18]:
# Step 1: Create a shortened mapping: first 5 letters -> Category
brands_df['short_name'] = brands_df['Name'].str[:5].str.lower()  # take first 5 letters, lowercase
name_to_category = dict(zip(brands_df['short_name'], brands_df['Category']))

# Step 2: Create a short_name column for df_filtered too
df_filtered['short_name'] = df_filtered['sponsor_user_full_name'].str[:5].str.lower()

# Step 3: Map based on short_name
df_filtered['Category'] = df_filtered['short_name'].map(name_to_category)

# Step 4: (Optional) Drop the helper column 'short_name'
df_filtered.drop(columns=['short_name'], inplace=True)

# Step 5: Check result
print(df_filtered[['sponsor_user_full_name', 'Category']].head())


             sponsor_user_full_name  \
122                    Vichy Brasil   
140  Maysa Leão ✨ Marketing Digital   
580                   Style at Home   
600                       ALLSAINTS   
710                            None   

                                        Category  
122  Personal Goods & General Merchandise Stores  
140                                          NaN  
580  Personal Goods & General Merchandise Stores  
600  Personal Goods & General Merchandise Stores  
710                                          NaN  


<ipython-input-18-901d3550d9f2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['short_name'] = df_filtered['sponsor_user_full_name'].str[:5].str.lower()
<ipython-input-18-901d3550d9f2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Category'] = df_filtered['short_name'].map(name_to_category)
<ipython-input-18-901d3550d9f2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

In [19]:
# Step 1: Mark sponsorship_final = 1 if influencer is in df_filtered
matched_usernames = set(df_filtered['owner.username'].dropna())
influencers_df['sponsorship_final'] = influencers_df['User'].apply(lambda x: 1 if x in matched_usernames else 0)

# Step 2: Prepare merge by dropping duplicates in df_filtered
merge_cols = ['owner.username', 'sponsor_user_username', 'sponsor_user_full_name', 'Category']
df_filtered_merge = df_filtered[merge_cols].drop_duplicates()

# Step 3: Merge into influencers_df on User == owner.username
merged_influencers = influencers_df.merge(df_filtered_merge, how='left', left_on='User', right_on='owner.username')

# Step 4: Drop redundant 'owner.username' column after merge
merged_influencers.drop(columns=['owner.username'], inplace=True)

# Step 5: View sample of result
print(merged_influencers.head())


                          Name  Followers Followees   Posts  \
0             • Silvia Pérez •    22899.0       760  2797.0   
1    ➕ Jodi ➕ Empowered Mom  ➕     1880.0      1935  1255.0   
2               rachelclindsay     1259.0      2867  1775.0   
3  CABELO - CACHOS - TRANSIÇÃO    72501.0      1198  2995.0   
4                   Sean 🙋🏽‍♂️     1896.0      1089   251.0   

                                                 URL              Category_x  \
0                   http://www.todoloquemellama.com/  Creators & Celebrities   
1            http://www.peppermintandgrapefruit.com/              Publishers   
2                                                NaN                     NaN   
3  http://api.whatsapp.com/send?1=pt_BR&phone=552...  Creators & Celebrities   
4                      http://www.mensmake-up.co.uk/                     NaN   

                                                 Bio  \
0                  Fashion | Lifestyle | Romeo's mom   
1  Essential Oils ➕ Boy Mama

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [20]:
# Filter and display only sponsored influencers
sponsored_influencers = merged_influencers[merged_influencers['sponsorship_final'] == 1]

# Show the result
print(sponsored_influencers)


                                Name  Followers Followees   Posts  \
5                      Amy Hopkinson    46736.0       754  2149.0   
23     Therese Charlotte Margrethe 🎀   247070.0       553  1626.0   
37                   Camilla Thurlow  1471030.0       188   320.0   
55          Jean | Extra Petite Blog   497538.0       240  2705.0   
85            Parenthood.Baby.Style.   112124.0      5407   792.0   
...                              ...        ...       ...     ...   
40014                    sarah tripp   384519.0       933  3960.0   
40015                    sarah tripp   384519.0       933  3960.0   
40030                    Poppy Deyes  1296390.0       385   503.0   
40058   🌸 N A T A L I E 🌸 sweet life    13864.0      1680   931.0   
40059   🌸 N A T A L I E 🌸 sweet life    13864.0      1680   931.0   

                                                     URL  \
5      https://www.justgiving.com/fundraising/amy-hop...   
23                                 http://tcmn.blog

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [ ]:
print(merged_influencers.columns)

In [21]:
# Drop the specified columns
mlpr = merged_influencers.drop(columns=['E-mail', 'Phone', 'Profile_pic', 'URL'], errors='ignore')

# Preview the updated DataFrame
print(mlpr.columns)

Index(['Name', 'Followers', 'Followees', 'Posts', 'Category_x', 'Bio', 'User',
       'sponsorship_final', 'sponsor_user_username', 'sponsor_user_full_name',
       'Category_y'],
      dtype='object')


In [ ]:
mlpr.head(100)

In [ ]:
print(df.columns)


In [22]:
df1=pd.read_csv('/kaggle/input/influencer-dataset/final_dataset.csv')

<ipython-input-22-93a2d898546f>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv('/kaggle/input/influencer-dataset/final_dataset.csv')


In [23]:
import pandas as pd

# Load dataset
jsondf = pd.read_csv('/kaggle/input/influencer-dataset/dataset.csv', usecols=[
    "taken_at_timestamp", "dimensions.height", "dimensions.width", 
    "edge_media_preview_like.count", "edge_media_to_caption.edges", 
    "edge_media_to_comment.count", "edge_media_to_comment.edges", 
    "edge_media_to_sponsor_user.edges", "edge_media_to_tagged_user.edges", 
    "location.name", "owner.full_name", "owner.is_verified", "owner.username"
])



<ipython-input-23-a2170fb54131>:4: DtypeWarning: Columns (8,11,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  jsondf = pd.read_csv('/kaggle/input/influencer-dataset/dataset.csv', usecols=[


In [24]:
print(jsondf.columns)

Index(['taken_at_timestamp', 'dimensions.height', 'dimensions.width',
       'edge_media_preview_like.count', 'edge_media_to_caption.edges',
       'edge_media_to_comment.count', 'edge_media_to_comment.edges',
       'edge_media_to_sponsor_user.edges', 'edge_media_to_tagged_user.edges',
       'location.name', 'owner.full_name', 'owner.is_verified',
       'owner.username'],
      dtype='object')


In [25]:
# Convert likes and comments columns to numeric, forcing errors to NaN
jsondf["edge_media_preview_like.count"] = pd.to_numeric(jsondf["edge_media_preview_like.count"], errors="coerce")
jsondf["edge_media_to_comment.count"] = pd.to_numeric(jsondf["edge_media_to_comment.count"], errors="coerce")

# Drop rows where likes or comments are NaN (removes non-numeric values)
jsondf = jsondf.dropna(subset=["edge_media_preview_like.count", "edge_media_to_comment.count"])



# Display first few rows
print(jsondf.head())


  taken_at_timestamp dimensions.height dimensions.width  \
0         1525437168               719             1080   
1         1529530145               750              750   
2         1542259188              1080             1080   
3         1509624930              1350             1080   
4         1521131737              1294             1080   

   edge_media_preview_like.count  \
0                          168.0   
1                         1160.0   
2                          220.0   
3                         1215.0   
4                          143.0   

                         edge_media_to_caption.edges  \
0  [{'node': {'text': 'Was nominated by @nick.f82...   
1  [{'node': {'text': 'Follow the movement and ge...   
2  [{'node': {'text': 'For quite a long time, I’v...   
3  [{'node': {'text': 'Still obsessed with these ...   
4  [{'node': {'text': 'Staying cozy wearing @tart...   

   edge_media_to_comment.count  \
0                         16.0   
1                      

In [26]:
# Forward fill usernames to copy the username down
jsondf["owner.username"] = jsondf["owner.username"].ffill()

# Print sample usernames to verify
print(jsondf[["owner.username"]].head(15))


     owner.username
0               NaN
1               NaN
2       essiesophie
3       essiesophie
4       essiesophie
5       essiesophie
6       essiesophie
7       essiesophie
8       essiesophie
9       ellaxarnold
10      ellaxarnold
11  madison_silotti
12  madison_silotti
13  madison_silotti
14  madison_silotti


In [27]:
jsondf["edge_media_preview_like.count"] = pd.to_numeric(jsondf["edge_media_preview_like.count"], errors="coerce")
jsondf["edge_media_to_comment.count"] = pd.to_numeric(jsondf["edge_media_to_comment.count"], errors="coerce")

# Print sample values to verify
print(jsondf[["edge_media_preview_like.count", "edge_media_to_comment.count"]].head(10))
nan_count = jsondf['edge_media_preview_like.count'].isna().sum()
print(f"📉 Number of NaN values in 'Average Engagement Rate': {nan_count}")

   edge_media_preview_like.count  edge_media_to_comment.count
0                          168.0                         16.0
1                         1160.0                          0.0
2                          220.0                         12.0
3                         1215.0                        103.0
4                          143.0                          4.0
5                          172.0                         11.0
6                           13.0                          2.0
7                         3416.0                         40.0
8                           20.0                          1.0
9                          335.0                         65.0
📉 Number of NaN values in 'Average Engagement Rate': 0


In [28]:
# Group by owner.username and calculate average of likes and comments
averaged_metrics = jsondf.groupby('owner.username')[['edge_media_preview_like.count', 'edge_media_to_comment.count']].mean().reset_index()

# Optionally rename columns for clarity
averaged_metrics.columns = ['owner.username', 'Avg Likes', 'Avg Comments']

# Show the result

averaged_metrics.shape


(41266, 3)

In [29]:
# Step 1: Calculate average likes and comments for each influencer from jsondf
avg_metrics_df = jsondf.groupby('owner.username')[['edge_media_preview_like.count', 'edge_media_to_comment.count']].mean().reset_index()

# Rename columns for clarity
avg_metrics_df.rename(columns={
    'owner.username': 'User',
    'edge_media_preview_like.count': 'Average Likes',
    'edge_media_to_comment.count': 'Average Comments'
}, inplace=True)

# Step 2: Merge with mlpr DataFrame on 'User'
mlpr = mlpr.merge(avg_metrics_df, on='User', how='left')

# Step 3: Display result
print("\n✅ mlpr with Average Likes & Comments added:\n")
print(mlpr[['User', 'Average Likes', 'Average Comments']].head(10))
# Count NaNs in Average Likes and Average Comments



✅ mlpr with Average Likes & Comments added:

                      User  Average Likes  Average Comments
0   silviatodoloquemellama    2194.941176         62.705882
1  peppermintandgrapefruit            NaN               NaN
2           rachelclindsay     748.909091         38.000000
3      jaquelinesantosblog    8036.333333        112.833333
4           skincarebysean            NaN               NaN
5              wellness_ed     442.785714         30.214286
6              susanearlam    4429.909091        133.818182
7                mrs___mel            NaN               NaN
8                 hh_dubai      46.000000          0.000000
9                     dele            NaN               NaN


/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [30]:
# Count NaNs in Average Likes and Average Comments
nan_likes = mlpr['Average Likes'].isna().sum()
nan_comments = mlpr['Average Comments'].isna().sum()

print(f"🔢 Number of NaN values in Average Likes: {nan_likes}")
print(f"🔢 Number of NaN values in Average Comments: {nan_comments}")


🔢 Number of NaN values in Average Likes: 14708
🔢 Number of NaN values in Average Comments: 14708


In [31]:
# Ensure all values are numeric and positive

mlpr['Average Likes'] = pd.to_numeric(mlpr['Average Likes'], errors='coerce')
mlpr['Average Comments'] = pd.to_numeric(mlpr['Average Comments'], errors='coerce')
mlpr['Followers'] = pd.to_numeric(mlpr['Followers'], errors='coerce')

# Filter out rows where Followers is zero or missing to avoid division errors
mlpr = mlpr[mlpr['Followers'] > 0]

# Calculate Engagement Rate (%)
mlpr['Engagement Rate'] = ((mlpr['Average Likes'] + 2*mlpr['Average Comments']) / mlpr['Followers']) * 100
# List of columns to be removed


/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


In [ ]:
mlpr.columns


In [ ]:
mlpr.head(10)

In [32]:
mlpr = mlpr.rename(columns={
    'Name': 'influencer_name',
    'Followers': 'Followers',
    'Followees': 'following',
    'Posts': 'No.of_posts',
    'Category_x': 'Category',
    'Bio': 'Bio',
    'User': 'User',
    'sponsorship_final': 'sponsorship_final',
    'sponsor_user_username': 'sponsor_user_username',
    'sponsor_user_full_name': 'sponsor_user_full_name',
    'Category_y': 'Brand Category',
    'Average Likes': 'Average Likes',
    'Average Comments': 'Average Comments',
    'Engagement Rate': 'Engagement Rate'
})

In [34]:
mlpr.columns
# Save DataFrame to CSV
mlpr.to_csv('mlpr.csv', index=False)


In [ ]:
brands_df
brands_df.head

In [35]:
brands_df_final = mlpr[['sponsor_user_username', 'sponsor_user_full_name', 'Brand Category']].dropna().drop_duplicates()
brands_df_final.columns = ['Brand Username', 'Brand Name', 'Brand Category']


In [36]:
brands_df_final

,Brand Username,Brand Name,Brand Category
5,pukkaherbs,Pukka Herbs,Grocery & Convenience Stores
37,disneyuk,Royal Air Force,Transportation & Accomodation Services
55,anntaylor,Ann Taylor,Personal Goods & General Merchandise Stores
85,wonderparkmovie,Parenthood.Baby.Style.,Creators & Celebrities
95,olay,Denise Bidot,Creators & Celebrities
...,...,...,...
39993,fabfitfun,Mark and Graham,Creators & Celebrities
40008,cetaphilus,Cetaphil US,Personal Goods & General Merchandise Stores
40009,walmart,Ellen,Creators & Celebrities
40010,loccitaneusa,LIKEtoKNOW.it,Personal Goods & General Merchandise Stores


In [37]:
brands_df_final = brands_df_final.merge(
    brands_df[['Name', 'Bio']],
    left_on='Brand Name',
    right_on='Name',
    how='left'
)

brands_df_final.drop(columns=['Name'], inplace=True)


In [39]:
brands_df_final
brands_df_final.to_csv('brands_df_final.csv', index=False)

In [40]:
influencers_df_final = mlpr[['User', 'influencer_name', 'Category', 'Followers', 'Average Likes', 'Average Comments', 'Engagement Rate', 'Bio']].drop_duplicates()
influencers_df_final.columns = ['User', 'Influencer Name', 'Influencer Category', 'Followers', 'Average Likes', 'Average Comments', 'Engagement Rate', 'Bio']


In [41]:
influencers_df_final
influencers_df_final.to_csv('influencers_df_final.csv', index=False)

In [ ]:
brands_df_final['key'] = 1
influencers_df_final['key'] = 1

# Cross join
big_df = pd.merge(brands_df_final, influencers_df_final, on='key').drop('key', axis=1)

print(big_df.shape)

In [ ]:
y_test_pred[0]

In [ ]:
y_train.value_counts()

In [ ]:
adi_df